## Setup
Load initial libraries and config to get things up an running.

In [ ]:
import os
import openai
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

openai.api_key  = os.environ['OPENAI_API_KEY']

template = """Answer the question based on the context below. Keep the answer short. Use two sentences maxiumum. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Helpful Answer:"""

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

vectordb = Chroma(
    persist_directory="../vectors/",
    embedding_function=OpenAIEmbeddings()
)

question_answerer = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    retriever=vectordb.as_retriever(),
    combine_docs_chain_kwargs={"prompt": PromptTemplate.from_template(template)},
    memory=memory
)

## Chatbot Layout
Setup an interface to interact with the chatbot.

In [ ]:
from IPython.display import HTML
from ipywidgets import widgets

def text_eventhandler(*event):
    if event[0]["new"] == "":
        return

    loading_bar.layout.display = "block"

    question = event[0]["new"]

    event[0]["owner"].value = ""

    question_chat_bubble = (
        f'<div class="chat-message-right pb-4"><div>'
        + f'<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">You</div>{question}</div>'
    )

    output.append_display_data(HTML(question_chat_bubble))

    try:
        response = question_answerer({"question": f"{question}"})
        answer = response["answer"]
    except Exception as e:
        answer = "<b>Error:</b> " + str(e)

    answer_chat_bubble = (
        f'<div class="chat-message-left pb-4"><div>'
        + f'<div class="flex-shrink-1 bg-primary text-white rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold text-white mb-1">Malina Bot</div>{answer}</div>'
    )

    loading_bar.layout.display = "none"

    output.append_display_data(HTML(answer_chat_bubble))

In [ ]:
%%html
<link rel="stylesheet" 
      href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" 
      integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" 
      crossorigin="anonymous">
<style>
    body {
        background-color: #21222B;
        margin-top:20px;
    }

    .bg-primary {
        background-color: #ee3f6e !important;
    }

    .chat-message-left,
    .chat-message-right {
        display: flex;
        flex-shrink: 0
    }

    .chat-message-left {
        margin-right: auto
    }

    .chat-message-right {
        flex-direction: row-reverse;
        margin-left: auto
    }
</style>

In [ ]:
from IPython.display import HTML, display

text_input = widgets.Text()
text_input.continuous_update = False
text_input.observe(text_eventhandler, "value")

output = widgets.Output()

load_image = open("../images/loading.gif", "rb")
loading_bar = widgets.Image(
    value=load_image.read(), 
    format="gif", 
    width="20", 
    height="20", 
    layout={ "display": "None" }
)

display(
    widgets.HBox(
        [output],
        layout=widgets.Layout(
            width="100%",
            display="inline-flex",
            flex_flow="column-reverse",
        ),
    )
)

display(
    widgets.Box(
        children=[loading_bar, text_input],
        layout=widgets.Layout(display="flex", flex_flow="row"),
    )
)